In [5]:
import numpy as np
import keras
from keras import layers
import pandas as pd
from sklearn.model_selection import train_test_split

# Load the CSV file
csv_file = "CIOD.csv"
data = pd.read_csv(csv_file)

# Split the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(data.iloc[:, :-1].values, data.iloc[:, -1].values, test_size=0.2, random_state=42)

input_shape = x_train.shape[1:]
print("Input_shape : ", input_shape)

x_train = x_train.reshape((x_train.shape[0], x_train.shape[1], 1))
x_test = x_test.reshape((x_test.shape[0], x_test.shape[1], 1))

input_shape = x_train.shape[1:]
print("Input_shape : ", input_shape)

# Calculate the number of unique classes
n_classes = len(np.unique(y_train))

def tmr_transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0):
    # Triple Encoders
    encoders_outputs = []
    for i in range(3):
        # Attention and Normalization
        encoder_input = inputs if i == 0 else encoders_outputs[i-1]
        x = layers.MultiHeadAttention(
            key_dim=head_size, num_heads=num_heads, dropout=dropout
        )(encoder_input, encoder_input)
        x = layers.Dropout(dropout)(x)
        x = layers.LayerNormalization(epsilon=1e-6)(x)
        res = x + encoder_input

        # Feed Forward Part
        x = layers.Conv1D(filters=ff_dim, kernel_size=1, activation="relu")(res)
        x = layers.Dropout(dropout)(x)
        x = layers.Conv1D(filters=inputs.shape[-1], kernel_size=1)(x)
        x = layers.LayerNormalization(epsilon=1e-6)(x)
        encoders_outputs.append(x + res)

    # Majority Voter (Decoder)
    voter_input = layers.Concatenate(axis=2)(encoders_outputs)
    voter_input = layers.Dense(inputs.shape[-1], activation='relu')(voter_input)
    voter_output = layers.Dense(inputs.shape[-1], activation='softmax')(voter_input)

    # Updating Encoders
    updated_encoders_outputs = []
    for i in range(3):
        x = layers.Add()([voter_output, encoders_outputs[i]])
        updated_encoders_outputs.append(x)

    return updated_encoders_outputs, voter_output

def build_model(
    input_shape,
    head_size,
    num_heads,
    ff_dim,
    num_transformer_blocks,
    mlp_units,
    n_classes,  
    dropout=0,
    mlp_dropout=0,
):
    inputs = keras.Input(shape=input_shape)
    x = inputs
    for _ in range(num_transformer_blocks):
        encoders_outputs, _ = tmr_transformer_encoder(x, head_size, num_heads, ff_dim, dropout)
        x = layers.Concatenate(axis=2)(encoders_outputs)

    x = layers.GlobalAveragePooling1D(data_format="channels_last")(x)
    for dim in mlp_units:
        x = layers.Dense(dim, activation="relu")(x)
        x = layers.Dropout(mlp_dropout)(x)
    outputs = layers.Dense(n_classes, activation="softmax")(x)
    return keras.Model(inputs, outputs)



    inputs = keras.Input(shape=input_shape)
    x = inputs
    for _ in range(num_transformer_blocks):
        x = transformer_encoder(x, head_size, num_heads, ff_dim, dropout)

    x = layers.GlobalAveragePooling1D(data_format="channels_last")(x)
    for dim in mlp_units:
        x = layers.Dense(dim, activation="relu")(x)
        x = layers.Dropout(mlp_dropout)(x)
    outputs = layers.Dense(n_classes, activation="softmax")(x)
    return keras.Model(inputs, outputs)

input_shape = x_train.shape[1:]

print("Input_shape : ", input_shape)

# Calculate the number of unique classes
n_classes = len(np.unique(y_train))

model = build_model(
    input_shape=input_shape,  # Specify input_shape as a keyword argument
    head_size=256,
    num_heads=4,
    ff_dim=4,
    num_transformer_blocks=4,
    mlp_units=[128],
    mlp_dropout=0.4,
    n_classes=n_classes,  
    dropout=0.25,
)

model.compile(
    loss="sparse_categorical_crossentropy",
    optimizer=keras.optimizers.Adam(learning_rate=1e-4),
    metrics=["sparse_categorical_accuracy"],
)
model.summary()

callbacks = [keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)]

model.fit(
    x_train,
    y_train,
    validation_split=0.2,
    epochs=10,
    batch_size=64,
    callbacks=callbacks,
)

model.evaluate(x_test, y_test, verbose=1)



Input_shape :  (45,)
Input_shape :  (45, 1)
Input_shape :  (45, 1)
Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 45, 1)]      0           []                               
                                                                                                  
 multi_head_attention_4 (MultiH  (None, 45, 1)       7169        ['input_2[0][0]',                
 eadAttention)                                                    'input_2[0][0]']                
                                                                                                  
 dropout_9 (Dropout)            (None, 45, 1)        0           ['multi_head_attention_4[0][0]'] 
                                                                                                  
 layer_normalization_8 (L

[6.923096656799316, 0.6286160945892334]

In [6]:
# Evaluate the model
evaluation_results = model.evaluate(x_test, y_test, verbose=1)

# Extract accuracy and other metrics
loss = evaluation_results[0]
accuracy = evaluation_results[1]

print("Test Loss:", loss)
print("Test Accuracy:", accuracy)


120/120 [==============================] - 19s 160ms/step - loss: 6.9231 - sparse_categorical_accuracy: 0.6286
Test Loss: 6.923096656799316
Test Accuracy: 0.6286160945892334
